### Code for data scraping and saving it into a csv 

In [ ]:
import requests
import pandas as pd
import mysql.connector
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import ast

# Set up the WebDriver for Selenium
driver = webdriver.Chrome()

# Function to log in to the website using Selenium
def login_with_selenium():
    driver.get('https://central1.recallmax.com/centralServer/request/admin/public/user/login.html')
    username = driver.find_element(By.ID, 'username')
    password = driver.find_element(By.ID, 'password')
    username.send_keys('username_revoked')
    password.send_keys('password_revoked')
    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')
    login_button.click()
    time.sleep(5)

# Function to extract cookies and use them in requests
def get_authenticated_session():
    selenium_cookies = driver.get_cookies()
    session = requests.Session()
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    return session

# Function to fetch all survey responses for a specific year
def get_survey_responses(session, year):
    url = "https://can5.recallmax.com/rsm/request/dash/secure/survey/surveyResponsesTable"
    params = {
        "accountId": 67640,
        "period": year,
        "_": int(time.time() * 1000)
    }
    response = session.get(url, params=params)
    if response.status_code == 200:
        try:
            return response.json().get("data", [])
        except ValueError:
            print(f"Failed to parse JSON response for year {year}.")
            return []
    else:
        print(f"Failed to fetch survey responses for year {year}. Status code: {response.status_code}")
        return []

# Function to fetch survey details for each survey
def get_survey_details(session, survey_id):
    url = "https://can5.recallmax.com/rsm/request/dash/secure/survey/surveyResponseDetails"
    params = {"accountId": 67640, "surveyId": survey_id}
    response = session.get(url, params=params)
    if response.status_code == 200:
        try:
            return response.json()
        except ValueError:
            print(f"Failed to parse survey details for survey ID {survey_id}.")
            return {}
    else:
        print(f"Failed to fetch details for survey ID {survey_id}. Status code: {response.status_code}")
        return {}

# Scrape, clean, and structure survey data into a DataFrame
def scrape_surveys():
    login_with_selenium()
    session = get_authenticated_session()
    all_survey_data = []

    for year in range(2019, 2025):
        survey_responses = get_survey_responses(session, year)
        if not survey_responses:
            continue
        for survey in survey_responses:
            survey_id = survey.get("surveyId")
            if survey_id:
                survey_details = get_survey_details(session, survey_id)
                if survey_details:
                    survey_data = {**survey, **survey_details, "year": year}
                    all_survey_data.append(survey_data)

    # Create and clean DataFrame
    df = pd.DataFrame(all_survey_data)
    df.drop(['contacted', 'unread', 'ratingRaw', 'apptDateRaw', 'emailHref', 'avgRating', 'selectedSurveyResponseId'], axis=1, inplace=True)
    df = df.replace(['', ' ', 'nan', 'NULL', 'N/A','NaN'], 'Null').fillna('None')

    def clean_apptProv(value):
        if isinstance(value, list) and value == ["N/A"]:
            return 'None'
        return value

    df['apptProv'] = df['apptProv'].apply(clean_apptProv)

    def clean_questionResponses(value):
        if isinstance(value, dict):
            return {k: ('NULL' if v in ['N/A', '', ' ','NULL','NaN'] else v) for k, v in value.items()}
        return value

    df['questionResponses'] = df['questionResponses'].apply(clean_questionResponses)
    responses_df = df['questionResponses'].apply(pd.Series)
    global updated_df 
    updated_df = pd.concat([df.drop(columns=['questionResponses']), responses_df], axis=1)
    updated_df.to_csv('C:/AMOD/finalscraped.csv')
    return updated_df

# Function to normalize provider names
def normalize_provider_name(provider):
    provider = re.sub(r'\[|\]', '', provider)
    provider = ' '.join(provider.split())
    return provider.strip()

# Database insertion functions
def insert_survey(cursor, survey_id, name, email, survey_date, appointment_date):
    cursor.execute("SELECT survey_id FROM surveys WHERE survey_id = %s", (survey_id,))
    result = cursor.fetchone()
    if result:
        return survey_id
    cursor.execute("INSERT INTO surveys (survey_id, name, email, survey_date, appointment_date) VALUES (%s, %s, %s, %s, %s)", (survey_id, name, email, survey_date, appointment_date))
    return survey_id

def insert_providers(cursor, provider_list):
    provider_ids = []
    if pd.notnull(provider_list):
        providers = [normalize_provider_name(p) for p in provider_list.split(', ')]
        for provider in providers:
            cursor.execute("SELECT id FROM providers WHERE name = %s", (provider,))
            result = cursor.fetchone()
            if result:
                provider_ids.append(result[0])
            else:
                cursor.execute("INSERT INTO providers (name) VALUES (%s)", (provider,))
                provider_ids.append(cursor.lastrowid)
    return provider_ids

def insert_response(cursor, survey_id, provider_id, comment):
    cursor.execute("SELECT id FROM responses WHERE survey_id = %s AND provider_id = %s", (survey_id, provider_id))
    result = cursor.fetchone()
    if result:
        cursor.execute("UPDATE responses SET comment = %s WHERE id = %s", (comment, result[0]))
        return result[0]
    cursor.execute("INSERT INTO responses (survey_id, provider_id, comment) VALUES (%s, %s, %s)", (survey_id, provider_id, comment))
    return cursor.lastrowid

def insert_question(cursor, question_text):
    cursor.execute("SELECT id FROM questions WHERE question_text = %s", (question_text,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO questions (question_text) VALUES (%s)", (question_text,))
    return cursor.lastrowid

def insert_response_detail(cursor, response_id, question_id, response_text):
    cursor.execute("SELECT id FROM response_details WHERE response_id = %s AND question_id = %s", (response_id, question_id))
    result = cursor.fetchone()
    if result:
        cursor.execute("UPDATE response_details SET response = %s WHERE id = %s", (response_text, result[0]))
    else:
        cursor.execute("INSERT INTO response_details (response_id, question_id, response) VALUES (%s, %s, %s)", (response_id, question_id, response_text))

# Main function to insert data into the database
def insert_data(cursor, df):
    question_columns = [col for col in df.columns if col not in ['contacted', 'ratingRaw', 'surveyId', 'apptDateRaw', 'name', 'rating', 'comment', 'apptDate', 'apptProv', 'email', 'emailHref', 'respDate', 'patientContacted', 'avgRating', 'selectedSurveyResponseId', 'year', 'Comments']]
    
    for _, row in df.iterrows():
        survey_id = insert_survey(cursor, row['surveyId'], row['name'], row['email'], row['respDate'], row['apptDate'])
        provider_ids = insert_providers(cursor, row['apptProv'])
        for provider_id in provider_ids:
            response_id = insert_response(cursor, survey_id, provider_id, row.get('Comments', None))
            for question in question_columns:
                question_id = insert_question(cursor, question)
                response_text = row[question]
                if pd.notnull(response_text):
                    insert_response_detail(cursor, response_id, question_id, str(response_text))

# Database connection and main process
conn = mysql.connector.connect(user='root', password='dentalclinic', host='localhost', database='dental_clinic')
cursor = conn.cursor()

try:
    scrape_surveys()
    df = pd.read_csv("C:/AMOD/finalscraped.csv")
    insert_data(cursor, df)
    conn.commit()
except Exception as e:
    conn.rollback()
    print("An error occurred:", e)
finally:
    cursor.close()
    conn.close()
    driver.quit()



### Cross-Validation to check if the data has been inserted into Mysql without any error

In [ ]:
import pandas as pd
import mysql.connector
from datetime import datetime
import re

# Load the CSV file for comparison
df = pd.read_csv('C:/AMOD/finalscraped.csv')

# Database connection setup
conn = mysql.connector.connect(user='root', password='dentalclinic', host='localhost', database='dental_clinic')
cursor = conn.cursor()

# Function to convert date format to 'YYYY-MM-DD'
def format_date(date_value):
    if pd.isna(date_value):
        return None
    return datetime.strptime(str(date_value), '%Y-%m-%d').date()

# Function to normalize provider names by removing special characters and extra spaces
def normalize_provider_name(provider):
    provider = re.sub(r'\[|\]', '', provider)  
    provider = ' '.join(provider.split()) 

# Test Case 1: Check Survey Record Count
def test_survey_count():
    unique_survey_ids = df['surveyId'].nunique()
    cursor.execute("SELECT COUNT(*) FROM surveys")
    db_count = cursor.fetchone()[0]
    assert unique_survey_ids == db_count, f"Survey count mismatch: CSV={unique_survey_ids}, DB={db_count}"

# Test Case 2: Check Provider Record Count
def test_provider_count():
    unique_providers = set(
        normalize_provider_name(provider.strip()) for providers in df['apptProv'].dropna() for provider in providers.split(', ')
    )
    cursor.execute("SELECT COUNT(*) FROM providers")
    db_count = cursor.fetchone()[0]
    assert len(unique_providers) == db_count, f"Provider count mismatch: CSV={len(unique_providers)}, DB={db_count}"

# Test Case 3: Check Response Record Count
def test_response_count():
    unique_responses = set()
    for _, row in df.iterrows():
        survey_id = row['surveyId']
        if pd.notnull(row['apptProv']):
            providers = row['apptProv'].split(', ')
            for provider in providers:
                provider_normalized = normalize_provider_name(provider.strip())
                unique_responses.add((survey_id, provider_normalized))
    
    cursor.execute("SELECT COUNT(*) FROM responses")
    db_count = cursor.fetchone()[0]
    assert len(unique_responses) == db_count, f"Response count mismatch: CSV={len(unique_responses)}, DB={db_count}"

# Test Case 4: Check Question Record Count
def test_question_count():
    question_columns = [
        col for col in df.columns if col not in [
            'contacted', 'ratingRaw', 'surveyId', 'apptDateRaw', 'name', 'rating', 'comment', 'apptDate',
            'apptProv', 'email', 'emailHref', 'respDate', 'patientContacted', 'avgRating',
            'selectedSurveyResponseId', 'year', 'Comments'
        ]
    ]
    cursor.execute("SELECT COUNT(*) FROM questions")
    db_count = cursor.fetchone()[0]
    assert len(question_columns) == db_count, f"Question count mismatch: CSV={len(question_columns)}, DB={db_count}"

# Test Case 5: Data Integrity Check with date format conversion
def test_data_integrity():
    # Check each surveyId entry from the CSV matches data in the DB
    for _, row in df.iterrows():
        survey_id = row['surveyId']
        
        # Fetch the record from the database
        cursor.execute("SELECT name, email, survey_date, appointment_date FROM surveys WHERE survey_id = %s", (survey_id,))
        result = cursor.fetchone()
        
        assert result is not None, f"SurveyId {survey_id} not found in DB"
        
        name, email, db_survey_date, db_appointment_date = result
        
        # Format the survey dates from both the CSV and the DB for comparison
        csv_survey_date = format_date(row['respDate'])
        csv_appointment_date = format_date(row['apptDate'])
        
        # Handle NULL email by checking if both are None (null in both CSV and DB)
        csv_email = row['email'] if pd.notna(row['email']) else None
        db_email = email if email is not None else None
        
        # Perform the assertions with the formatted dates
        assert name == row['name'], f"Name mismatch for SurveyId {survey_id}"
        assert csv_email == db_email, f"Email mismatch for SurveyId {survey_id}: CSV={csv_email}, DB={db_email}"
        assert db_survey_date == csv_survey_date, f"Survey Date mismatch for SurveyId {survey_id}: DB={db_survey_date}, CSV={csv_survey_date}"
        assert db_appointment_date == csv_appointment_date, f"Appointment Date mismatch for SurveyId {survey_id}: DB={db_appointment_date}, CSV={csv_appointment_date}"

try:
    test_survey_count()
    test_provider_count()
    test_response_count()
    test_question_count()
    test_data_integrity()
    print("All tests passed successfully!")
except AssertionError as e:
    print("Test failed:", e)
finally:
    cursor.close()
    conn.close()


###  Data cleaning for power bi

In [ ]:
import pandas as pd
import re

# Read the data
data = pd.read_csv("C:/AMOD/finalscraped.csv")
df = pd.DataFrame(data)

def extract_roles(apptProv):
    if not isinstance(apptProv, str):
        apptProv = str(apptProv) 
    
    items = re.findall(r"['\"](.*?)['\"]", apptProv)
    
    dentist = None
    hygienist = None
    
    for item in items:
        if 'Dr.' in item and dentist is None:
            dentist = item.split('-')[0].strip()  
        elif 'Temp Hygiene' not in item and 'Not Available' not in item and hygienist is None:
            hygienist = item.split('-')[0].strip()
    
    return pd.Series([dentist, hygienist])

df[['dentist', 'hygienist']] = df['apptProv'].apply(extract_roles)

df = df.drop(columns=['apptProv'])

df.to_csv('C:/AMOD/finalscraped1.csv', index=False)

print("Processing complete. Data saved to 'finalscraped1.csv'.")


### Imputing missing values

In [ ]:
import pandas as pd
import numpy as np

# Load your data
data = pd.read_csv("C:/AMOD/finalscraped1.csv")
df = pd.DataFrame(data)

# List of columns that need to be imputed (excluding the ones for dentist and hygienist ratings)
question_columns = [
    "Upon arriving at our dental practice, how satisfied were you with our greeting?",
    "How satisfied were you with the amount of time spent in our waiting room?",
    "Please rate your comfort level during your appointment?",
    "How satisfied were you with your dental treatment options?",
    "How likely are you to refer your friends and family to our dental office?"
]

# Impute missing values with column averages (excluding dentist/hygienist columns)
for col in question_columns:
    df[col].fillna(df[col].mean(), inplace=True)

# Function to get dentist/hygienist average ratings
def impute_with_professional_rating(row, professional_type='dentist'):
    professional_column = 'dentist' if professional_type == 'dentist' else 'hygienist'
    professional = row[professional_column]
    
    if pd.notna(professional):
        professional_rating_column = f'How would you rate your {professional_type}?'
        professional_avg_rating = df[df[professional_column] == professional][professional_rating_column].mean()
        return professional_avg_rating
    else:
        return df[f'How would you rate your {professional_type}?'].mean()

# Impute missing values in "How would you rate your dentist?" and "How would you rate your hygienist?"
for index, row in df.iterrows():
    if pd.isna(row["How would you rate your dentist?"]):
        df.at[index, "How would you rate your dentist?"] = impute_with_professional_rating(row, 'dentist')
    
    if pd.isna(row["How would you rate your hygienist?"]):
        df.at[index, "How would you rate your hygienist?"] = impute_with_professional_rating(row, 'hygienist')

df.to_csv('C:/AMOD/finalscraped_imputed.csv', index=False)

print("Imputation complete. Data saved to 'finalscraped_imputed.csv'.")


### Sentimental Analysis using VADER

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
import pandas as pd

df = pd.read_csv("C:/AMOD/finalscraped_imputed.csv")
df = df[["surveyId","Comments"]]
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def vader_sentiment(comment):
    sentiment = sia.polarity_scores(str(comment)) 
    return sentiment['compound']

df['Sentiment Score'] = df['Comments'].apply(vader_sentiment)

def classify_sentiment_vader(score):
    if score > 0.05:
        return "Positive"
    elif score < -0.05:
        return "Negative"
    else:
        return "Neutral"

df['Sentiment Label'] = df['Sentiment Score'].apply(classify_sentiment_vader)

print(df[['Comments', 'Sentiment Score', 'Sentiment Label']])
df.to_csv("C:/AMOD/Sentiments.csv")
